<a href="https://colab.research.google.com/github/adamMcneil/cs-440-final-project/blob/main/Adam_CS440.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
import time
!pip install datasets
from datasets import load_dataset

from transformers import AutoTokenizer, LlamaForSequenceClassification
import torch

from peft import get_peft_model, LoraConfig, TaskType
!pip install transformers
from transformers import Trainer, TrainingArguments

from transformers import BertTokenizer, BertForSequenceClassification

In [44]:
model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
# Helper function to classify one example
def classify_sentiment(test_model, review_text):
    # Tokenize
    inputs = bert_tokenizer(review_text, return_tensors="pt", padding=False, truncation=True).to(device)

    # Forward pass (no gradient needed during inference)
    with torch.no_grad():
        outputs = test_model(**inputs)
        logits = outputs.logits

    # Predicted class
    predicted_class_id = logits.argmax(dim=-1).item()

    # (Optional) Map class ID to label
    id2label = test_model.config.id2label
    predicted_label = id2label[predicted_class_id]

    #if predicted_label == "LABEL_0":
    #    predicted_label = 0
    #else:
    #    predicted_label = 1
    predicted_label = int(predicted_label.split("_")[-1])

    return predicted_label

In [58]:
def test_model(test_model, validation_dataset):
    print("Running classification test...")
    correct = 0
    total = 0
    start_time = time.time()

    for example in validation_dataset:
        review_text = example['sentence']
        true_label = example['label']

        pred_label = classify_sentiment(test_model, review_text)

        if pred_label == true_label:
            correct += 1
        total += 1

    accuracy = correct / total
    elapsed_time = time.time() - start_time

    # Results
    print(f"\nAccuracy on {total} examples: {accuracy*100:.2f}%")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    print(f"Average time per sample: {elapsed_time/total:.2f} seconds")


In [24]:
# Testing
test_model(model)

Running classification test...

Accuracy on 872 examples: 49.66%
Time elapsed: 30.80 seconds
Average time per sample: 0.04 seconds


In [65]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
lora_model = get_peft_model(model, lora_config)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
bert_lora_model = get_peft_model(bert_model, lora_config)

dataset = load_dataset("stanfordnlp/sst2")
small_train = dataset["train"].select(range(5000))
small_val = dataset["validation"].select(range(400))
validation_dataset, small_train, small_val = load_ag_news()

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def preprocess(example):
    return bert_tokenizer(
        example["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
small_train = small_train.map(preprocess, batched=True)
small_val =  small_val.map(preprocess, batched=True)

bert_train = small_train.map(preprocess, batched=True)
bert_val = small_val.map(preprocess, batched=True)

training_args = TrainingArguments(
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

KeyError: 'sentence'

In [48]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val,
)
trainer.train()

test_model(lora_model, validation_dataset)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


KeyboardInterrupt: 

In [55]:
trainer = Trainer(
    model=bert_lora_model,
    args=training_args,
    train_dataset=bert_train,
    eval_dataset=bert_val,
)
trainer.train()

test_model(bert_lora_model, validation_dataset)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.661300
200,0.423600
300,0.328500


Running classification test...

Accuracy on 872 examples: 84.98%
Time elapsed: 20.66 seconds
Average time per sample: 0.02 seconds


In [13]:
test_model(lora_model, trials=100)

Running classification test...

Accuracy on 100 examples: 57.00%
Time elapsed: 6.57 seconds
Average time per sample: 0.07 seconds


In [66]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
# Helper function to classify one example
def classify_sentiment(test_model, review_text):
    # Tokenize
    inputs = tokenizer(review_text, return_tensors="pt", padding=False, truncation=True).to(device)

    # Forward pass (no gradient needed during inference)
    with torch.no_grad():
        outputs = test_model(**inputs)
        logits = outputs.logits

    # Predicted class
    predicted_class_id = logits.argmax(dim=-1).item()

    # (Optional) Map class ID to label
    id2label = test_model.config.id2label
    predicted_label = id2label[predicted_class_id]

    predicted_label = int(predicted_label.split("_")[-1])

    return predicted_label

def test_model(test_model, validation_dataset):
    print("Running classification test...")
    correct = 0
    total = 0
    start_time = time.time()

    for example in validation_dataset:
        review_text = example['text']
        true_label = example['label']

        pred_label = classify_sentiment(test_model, review_text)

        if pred_label == true_label:
            correct += 1
        total += 1

    accuracy = correct / total
    elapsed_time = time.time() - start_time

    # Results
    print(f"\nAccuracy on {total} examples: {accuracy*100:.2f}%")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    print(f"Average time per sample: {elapsed_time/total:.2f} seconds")


In [83]:
def load_ag_news():
    dataset = load_dataset("ag_news")
    small_train = dataset["train"].select(range(15000))
    small_val = dataset["test"].select(range(1000))
    return dataset["test"], small_train, small_val

In [78]:
validation_dataset, small_train, small_val = load_ag_news()

test_model(model, validation_dataset.select(range(1000)))

Running classification test...

Accuracy on 1000 examples: 12.60%
Time elapsed: 49.86 seconds
Average time per sample: 0.05 seconds


In [84]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
lora_model = get_peft_model(model, lora_config)

validation_dataset, small_train, small_val = load_ag_news()

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def preprocess(example):
    return bert_tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
small_train = small_train.map(preprocess, batched=True)
small_val =  small_val.map(preprocess, batched=True)

training_args = TrainingArguments(
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val,
)
trainer.train()

test_model(lora_model, validation_dataset.select(range(1000)))

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.291400
200,1.026800
300,0.914600
400,0.783200
500,0.709400
600,0.691500


In [81]:
test_model(lora_model, validation_dataset.select(range(1000)))

Running classification test...

Accuracy on 1000 examples: 18.90%
Time elapsed: 41.36 seconds
Average time per sample: 0.04 seconds
